In [1]:
import sys, os

from h5flow.data import dereference

layout_file = 'geo/layout-2.4.0.yaml'

vdrift = 1.5136504594138773 # mm/us for 0.5kV/cm

In [2]:
from collections import defaultdict
import yaml
import numpy as np

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
from collections import defaultdict

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

import matplotlib.pyplot as plt
import glob
import itertools

In [3]:
def load_tile_geometry(fpath):
    '''
    Build a dictionary for position lookup from tile layout.
 
    Arguments
    ---------
    fpath : str
        file path to the tile layout
    
    Returns
    -------
    geometry : dict
        dictionary of tile position in mm
        `geometry[(chip, channel)] -> (x,y,z)`   
    '''
    with open(fpath, 'r') as f:
        geometry_yaml = yaml.load(f, Loader=yaml.FullLoader)
          
    chips = geometry_yaml["chips"]
    pixels = geometry_yaml["pixels"]

    geometry = defaultdict(lambda: (None, None, None))
    
    for chip, channels in chips:
        for channel_id, channel in enumerate(channels):
            if channel is not None:
                x = pixels[channel][1]
                y = pixels[channel][2]
                z = 0
                geometry[(chip, channel_id)] = x, y, z

    return dict(geometry)


geom = load_tile_geometry(layout_file)

In [4]:
## size of the detector in [mm]
#### set boundary at the edge of the LArPix, not the inner boundary of the fieldshell
#### set z boundary loosely at 0, 300 mm
pixel_pitch = 4.43399

pos = np.array(list(geom.values())) 
x_min = pos[:, 0].min() - 0.5 * pixel_pitch
x_max = pos[:, 0].max() + 0.5 * pixel_pitch
abs_x_max = abs(pos[:, 0].max() + 0.5 * pixel_pitch)

y_min = pos[:, 1].min() - 0.5 * pixel_pitch
y_max = pos[:, 1].max() + 0.5 * pixel_pitch  
abs_y_max = abs(pos[:, 1].max() + 0.5 * pixel_pitch)

z_min = 0
z_max = 300


In [8]:
################
# Not very elegant
################

cathode_crosser = True

## if the earliest track start is more than 5mm away from the anode, shift all track ends in the event
z_shift_threshold = 5 # mm 
## if the track ends are within this threshold to the wall, consider they cross
## use the same threshold to exclude the tracks which show ambiguity on which side they cross
crossing_threshold = 2 * pixel_pitch # mm

## output: xmin, xmax, ymin, ymax, zmin, zmax
ends_at_edges = [[],[],[],[],[],[]]

## output: purity check
data_time = [] # one unix time stamp (middle point) per file
dQdx_arr = [] #mV/cm, one array entry per file
tdrift_arr = [] # one array entry per file


## dates
dates = ['2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05']
# dates = ['test']
for date in dates:
    directory = f"/sdf/group/neutrino/larpix/evd/selftrigger/{date}/"

    ## loop over all the files
    for filename in os.scandir(directory):
        if filename.is_file():
            f = h5py.File(filename.path, 'r')
#             print(filename.path)

            ## getting the event
            events = f['charge/events/data']
            
            ## time stamp of the file
            data_time.append(0.5 * (min(events['unix_ts']) + max(events['unix_ts'])))
            
            ## dQdx, drift_t for this file
            this_f_dQdx = np.array([])
            this_f_tdrift = np.array([])
            
            ## loop over the events in a file
            for i_ev in np.unique(events['id']):  

                ## get all the tracks
                tracks_ev = dereference(i_ev, ref=f['charge/events/ref/combined/tracklets/ref'], region=f['charge/events/ref/combined/tracklets/ref_region'], data=f['combined/tracklets/data'])
                trk_pool = tracks_ev['id'][0]

                ## loop over the tracks in an event
                #### in this iteration, we only care about track ends
                #### we may shift the tracks to start at the anode
                #### and we use the same shift for everything in the same event
                #### the shift should be extracted from the first track (closest to the anode)
                #### this version has not considered broken tracks yet
                track_ends_arr = np.array([])

                if np.ma.is_masked(trk_pool):
                    continue

                for i_trk, trk_id in enumerate(trk_pool):

                    ## get this track
                    mask = tracks_ev['id'] == trk_id
                    this_trk = tracks_ev[mask]
                
                    #### track_ends_arr [#tracks[[end_closer_to_the_anode xyz], [the other end xyz]]]
                    if this_trk['start'][:,2] < this_trk['end'][:,2]:
                        track_ends_temp = np.array([np.append(np.array(this_trk['start']), np.array(this_trk['end']), axis=0)])
                    else:
                        track_ends_temp = np.array([np.append(np.array(this_trk['end']), np.array(this_trk['start']), axis=0)])

                    if i_trk == 0:
                        track_ends_arr = track_ends_temp
                    else:
                        track_ends_arr = np.append(track_ends_arr, track_ends_temp, axis=0)

                ## shift tracks if the closest one is Xmm away from the anode
                z_shift = track_ends_arr[:,0,2].min()
                if z_shift > z_shift_threshold:
                    track_ends_arr[:, :, 2] = track_ends_arr[:, :, 2] - z_shift

                #### select track ends which are close to the wall
                #### one of the end needs to be close to the anode
                for i_trk in range(len(track_ends_arr)):
                        
                    ## get this track
                    ## the length and the order of the track should be the same
                    mask = tracks_ev['id'] == trk_pool[i_trk]
                    this_trk = tracks_ev[mask]
                    
                    ## first take out tracks across the whole length (diff_z > z_max - crossing_threshold)
                    if abs(track_ends_arr[i_trk, 1, 2] - track_ends_arr[i_trk, 0, 2]) > z_max - crossing_threshold:
                        ## the first end is closer to the anode by construction
                        if track_ends_arr[i_trk, 0, 2] > z_shift_threshold:
                            track_ends_arr[i_trk, 0, 2] = 0
                            track_ends_arr[i_trk, 1, 2] = track_ends_arr[i_trk, 1, 2] - track_ends_arr[i_trk, 0, 2]
                        ## anode side zmin
                        ends_at_edges[4].append(track_ends_arr[i_trk, 0])
                        ## cathode side zmin
                        ends_at_edges[5].append(track_ends_arr[i_trk, 1])
                        
                        continue

                    ## initialize the boolean
                    #### end1 and 2; xmin,max, ymin,max, zmin,max
                    end_edge = np.full((2,3,2),False)
                    for i_end in range(2):

                        ## x
                        if track_ends_arr[i_trk, i_end, 0] < x_min + crossing_threshold:
                            end_edge[i_end, 0, 0] = True
                        if track_ends_arr[i_trk, i_end, 0] > x_max - crossing_threshold:
                            end_edge[i_end, 0, 1] = True

                        ## y
                        if track_ends_arr[i_trk, i_end, 1] < y_min + crossing_threshold:
                            end_edge[i_end, 1, 0] = True
                        if track_ends_arr[i_trk, i_end, 1] > y_max - crossing_threshold:
                            end_edge[i_end, 1, 1] = True

                        ## z
                        if track_ends_arr[i_trk, i_end, 2] < z_min + crossing_threshold:
                            end_edge[i_end, 2, 0] = True
                        if track_ends_arr[i_trk, i_end, 2] > z_max - crossing_threshold:
                            end_edge[i_end, 2, 1] = True

                    ## collapse in min and max, give True if any of them is True 
                    end_edge_xyz = end_edge[:, :, 0] + end_edge[:, :, 1]

                    
                    ##################
                    ## if neither end is close to the boundary, skip
                    if any(end_edge_xyz[0]) == False and any(end_edge_xyz[1]) == False:
                        continue

                    ##################
                    ## if both ends are close to the boundary:
                    if any(end_edge_xyz[0]) == True and any(end_edge_xyz[1]) == True:

                        ## if any end is close to more than one wall, skip to avoid ambiguity
                        ## this excludes a scenario that 
                        ## if a track has wrong time, end1/2 is behind the cathode but also touch a side wall
                        ## for simplicity, they are not considered
                        continue_flag = False                        
                        for i_end in range(2):
                            if (end_edge_xyz[i_end][0] and end_edge_xyz[i_end][1]) or (end_edge_xyz[i_end][0] and end_edge_xyz[i_end][2]) or (end_edge_xyz[i_end][1] and end_edge_xyz[i_end][2]) == True:
                                continue_flag = True
                                continue
                        if continue_flag:
                            continue 
                            
                        ## if both ends are over the cathode, skip
                        if end_edge[0,2,1] and end_edge[1,2,1]:
                            continue
                                        
                        end1_edge_arr = np.expand_dims(end_edge[0].flatten(), axis = 1)        
                        end2_edge_arr = np.expand_dims(end_edge[1].flatten(), axis = 0) 
                        idx_true = np.where(end1_edge_arr * end2_edge_arr == True)
                        
                        ## end2 is closer to the cathode than end1 by construction
                        ## 5 -> zmax (cathode)
                        if idx_true[1][0] == 5 and track_ends_arr[i_trk, 1, 2] > z_max + crossing_threshold * 2:
                            if cathode_crosser:
                                track_ends_arr[i_trk, 0, 2] = track_ends_arr[i_trk, 0, 2] - (track_ends_arr[i_trk, 1, 2] - z_max)
                                track_ends_arr[i_trk, 1, 2] = z_max
                            else:
                                continue

                        for i_end in range(2):
                            ends_at_edges[idx_true[i_end][0]].append(track_ends_arr[i_trk, i_end])
                        
                        ## dQdx and drift z
                        dq = this_trk['dq'][0] * 0.221 # mV->ke-
                        this_dx = this_trk['dx'][0]
                        dx = np.linalg.norm(this_dx, axis=-1)
                        dQdx = dq[dx>0]/dx[dx>0] 
                        this_f_dQdx = np.append(this_f_dQdx, dQdx, axis=0)
                        
                        traj = this_trk['trajectory'][0][:-1]
                        traj_z = traj[:, 2]
                        traj_z_shift = track_ends_arr[i_trk, 0, 2] - min(traj_z)
                        z_start = traj_z + traj_z_shift
                        dz = this_dx[:, 2]
                        tdrift = (z_start[dx > 0] + dz[dx > 0] / 2) / vdrift
                        this_f_tdrift = np.append(this_f_tdrift, tdrift, axis=0)
                        
                        continue
                            
                    ##################
                    ## if only one of the end is close to the boundary,
                    ## it has to be end1 close to a side wall or a side wall and the anode
                    ## the other cases are ignored
                    if cathode_crosser:
                        if any(end_edge_xyz[0]) == True:
                            ## if end1 close to both x and y boundary, skip
                            ## if end1 close to neither x and y boundary, skip
                            if end_edge_xyz[0][0] == end_edge_xyz[0][1]:
                                continue
                            ## Now move the track so end2 touches the cathode
                            track_ends_arr[i_trk, 0, 2] = track_ends_arr[i_trk, 0, 2] - (track_ends_arr[i_trk, 1, 2] - z_max)
                            track_ends_arr[i_trk, 1, 2] = z_max
                          
                            idx_end1_true = np.where(end_edge[0].flatten() == True)

                            ends_at_edges[idx_end1_true[0][0]].append(track_ends_arr[i_trk, 0])
                            ends_at_edges[5].append(track_ends_arr[i_trk, 1])
                            
                            ## dQdx and drift time
                            dq = this_trk['dq'][0] * 0.221 # mV->ke-
                            this_dx = this_trk['dx'][0]
                            dx = np.linalg.norm(this_dx, axis=-1)
                            dQdx = dq[dx>0]/dx[dx>0]
                            this_f_dQdx = np.append(this_f_dQdx, dQdx, axis=0)

                            traj = this_trk['trajectory'][0][:-1]
                            traj_z = traj[:, 2]
                            traj_z_shift = track_ends_arr[i_trk, 0, 2] - min(traj_z)
                            z_start = traj_z + traj_z_shift
                            dz = this_dx[:, 2]
                            tdrift = (z_start[dx > 0] + dz[dx > 0] / 2) / vdrift
                            this_f_tdrift = np.append(this_f_tdrift, tdrift, axis=0)
                        
            dQdx_arr.append(this_f_dQdx)
            tdrift_arr.append(this_f_tdrift)
            
## track ends info                                
for i in range(6):
    ends_at_edges[i] = np.array(ends_at_edges[i])

ends_at_edges = np.array(ends_at_edges)
# print(ends_at_edges)
np.save(f"data/2022-08/track_ends_edge_22-07_22-08_cathode_crossers_{cathode_crosser}.npy", ends_at_edges)   

## purity info
dQdx_arr = np.array(dQdx_arr)            
tdrift_arr = np.array(tdrift_arr)
data_time = np.array(data_time)
np.savez(f"data/2022-08/track_purity_info_22-07_22-08_cathode_crossers_{cathode_crosser}.npz", data_time=data_time, dQdx=dQdx_arr, tdrift=tdrift_arr)   
 
  

/tmp/ipykernel_78570/3493950909.py:232: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ends_at_edges = np.array(ends_at_edges)
/tmp/ipykernel_78570/3493950909.py:237: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dQdx_arr = np.array(dQdx_arr)
/tmp/ipykernel_78570/3493950909.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tdrift

In [29]:
data = np.load(f"data/2022-08/track_purity_info_22-07_22-08_cathode_crossers_{cathode_crosser}.npz",allow_pickle=True)   
print(len(data['data_time']))
print(len(data['dQdx']))
print(len(data['tdrift']))

6
6
6


In [6]:
###############
# Backup for now
###############
################
# Not very elegant
################

cathode_crosser = False

## if the earliest track start is more than 5mm away from the anode, shift all track ends in the event
z_shift_threshold = 5 # mm 
## if the track ends are within this threshold to the wall, consider they cross
## use the same threshold to exclude the tracks which show ambiguity on which side they cross
crossing_threshold = 2 * pixel_pitch # mm

## output: xmin, xmax, ymin, ymax, zmin, zmax
ends_at_edges = [[],[],[],[],[],[]]

## dates
dates = ['2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05']
# dates = ['test']
for date in dates:
    directory = f"/sdf/group/neutrino/larpix/evd/selftrigger/{date}/"

    ## loop over all the files
    for filename in os.scandir(directory):
        if filename.is_file():
            f = h5py.File(filename.path, 'r')
#             print(filename.path)

            ## loop over the events in a file
            events = f['charge/events/data']
            for i_ev in np.unique(events['id']):  

                ## get all the tracks
                tracks_ev = dereference(i_ev, ref=f['charge/events/ref/combined/tracklets/ref'], region=f['charge/events/ref/combined/tracklets/ref_region'], data=f['combined/tracklets/data'])
                trk_pool = tracks_ev['id'][0]

                ## loop over the tracks in an event
                #### in this iteration, we only care about track ends
                #### we may shift the tracks to start at the anode
                #### and we use the same shift for everything in the same event
                #### the shift should be extracted from the first track (closest to the anode)
                #### this version has not considered broken tracks yet
                track_ends_arr = np.array([])

                if np.ma.is_masked(trk_pool):
                    continue

                for i_trk, trk_id in enumerate(trk_pool):

                    ## get this track
                    mask = tracks_ev['id'] == trk_id
                    this_trk = tracks_ev[mask]
                
                    #### track_ends_arr [#tracks[[end_closer_to_the_anode xyz], [the other end xyz]]]
                    if this_trk['start'][:,2] < this_trk['end'][:,2]:
                        track_ends_temp = np.array([np.append(np.array(this_trk['start']), np.array(this_trk['end']), axis=0)])
                    else:
                        track_ends_temp = np.array([np.append(np.array(this_trk['end']), np.array(this_trk['start']), axis=0)])

                    if i_trk == 0:
                        track_ends_arr = track_ends_temp
                    else:
                        track_ends_arr = np.append(track_ends_arr, track_ends_temp, axis=0)

                ## shift tracks if the closest one is Xmm away from the anode
                z_shift = track_ends_arr[:,0,2].min()
                if z_shift > z_shift_threshold:
                    track_ends_arr[:, :, 2] = track_ends_arr[:, :, 2] - z_shift

                #### select track ends which are close to the wall
                #### one of the end needs to be close to the anode
                for i_trk in range(len(track_ends_arr)):
                    
                    ## first take out tracks across the whole length (diff_z > z_max - crossing_threshold)
                    if abs(track_ends_arr[i_trk, 1, 2] - track_ends_arr[i_trk, 0, 2]) > z_max - crossing_threshold:
                        ## the first end is closer to the anode by construction
                        if track_ends_arr[i_trk, 0, 2] > z_shift_threshold:
                            track_ends_arr[i_trk, 0, 2] = 0
                            track_ends_arr[i_trk, 1, 2] = track_ends_arr[i_trk, 1, 2] - track_ends_arr[i_trk, 0, 2]
                        ## anode side zmin
                        ends_at_edges[4].append(track_ends_arr[i_trk, 0])
                        ## cathode side zmin
                        ends_at_edges[5].append(track_ends_arr[i_trk, 1])
                        
                        continue

                    ## initialize the boolean
                    #### end1 and 2; xmin,max, ymin,max, zmin,max
                    end_edge = np.full((2,3,2),False)
                    for i_end in range(2):

                        ## x
                        if track_ends_arr[i_trk, i_end, 0] < x_min + crossing_threshold:
                            end_edge[i_end, 0, 0] = True
                        if track_ends_arr[i_trk, i_end, 0] > x_max - crossing_threshold:
                            end_edge[i_end, 0, 1] = True

                        ## y
                        if track_ends_arr[i_trk, i_end, 1] < y_min + crossing_threshold:
                            end_edge[i_end, 1, 0] = True
                        if track_ends_arr[i_trk, i_end, 1] > y_max - crossing_threshold:
                            end_edge[i_end, 1, 1] = True

                        ## z
                        if track_ends_arr[i_trk, i_end, 2] < z_min + crossing_threshold:
                            end_edge[i_end, 2, 0] = True
                        if track_ends_arr[i_trk, i_end, 2] > z_max - crossing_threshold:
                            end_edge[i_end, 2, 1] = True

                    ## collapse in min and max, give True if any of them is True 
                    end_edge_xyz = end_edge[:, :, 0] + end_edge[:, :, 1]

                    
                    ##################
                    ## if neither end is close to the boundary, skip
                    if any(end_edge_xyz[0]) == False and any(end_edge_xyz[1]) == False:
                        continue

                    ##################
                    ## if both ends are close to the boundary:
                    if any(end_edge_xyz[0]) == True and any(end_edge_xyz[1]) == True:

                        ## if any end is close to more than one wall, skip to avoid ambiguity
                        ## this excludes a scenario that 
                        ## if a track has wrong time, end1/2 is behind the cathode but also touch a side wall
                        ## for simplicity, they are not considered
                        continue_flag = False                        
                        for i_end in range(2):
                            if (end_edge_xyz[i_end][0] and end_edge_xyz[i_end][1]) or (end_edge_xyz[i_end][0] and end_edge_xyz[i_end][2]) or (end_edge_xyz[i_end][1] and end_edge_xyz[i_end][2]) == True:
                                continue_flag = True
                                continue
                        if continue_flag:
                            continue 
                            
                        ## if both ends are over the cathode, skip
                        if end_edge[0,2,1] and end_edge[1,2,1]:
                            continue
                                        
                        end1_edge_arr = np.expand_dims(end_edge[0].flatten(), axis = 1)        
                        end2_edge_arr = np.expand_dims(end_edge[1].flatten(), axis = 0) 
                        idx_true = np.where(end1_edge_arr * end2_edge_arr == True)
                        
                        ## end2 is closer to the cathode than end1 by construction
                        ## 5 -> zmax (cathode)
                        if idx_true[1][0] == 5 and track_ends_arr[i_trk, 1, 2] > z_max + crossing_threshold * 2:
                            if cathode_crosser:
                                track_ends_arr[i_trk, 0, 2] = track_ends_arr[i_trk, 0, 2] - (track_ends_arr[i_trk, 1, 2] - z_max)
                                track_ends_arr[i_trk, 1, 2] = z_max
                            else:
                                continue

                        for i_end in range(2):
                            ends_at_edges[idx_true[i_end][0]].append(track_ends_arr[i_trk, i_end])
                        
                        continue
                            
                    ##################
                    ## if only one of the end is close to the boundary,
                    ## it has to be end1 close to a side wall or a side wall and the anode
                    ## the other cases are ignored
                    if cathode_crosser:
                        if any(end_edge_xyz[0]) == True:
                            ## if end1 close to both x and y boundary, skip
                            ## if end1 close to neither x and y boundary, skip
                            if end_edge_xyz[0][0] == end_edge_xyz[0][1]:
                                continue
                            ## Now move the track so end2 touches the cathode
                            track_ends_arr[i_trk, 0, 2] = track_ends_arr[i_trk, 0, 2] - (track_ends_arr[i_trk, 1, 2] - z_max)
                            track_ends_arr[i_trk, 1, 2] = z_max
                          
                            idx_end1_true = np.where(end_edge[0].flatten() == True)

                            ends_at_edges[idx_end1_true[0][0]].append(track_ends_arr[i_trk, 0])
                            ends_at_edges[5].append(track_ends_arr[i_trk, 1])
                      
                                
for i in range(6):
    ends_at_edges[i] = np.array(ends_at_edges[i])

ends_at_edges = np.array(ends_at_edges)
print(ends_at_edges)
np.save(f"data/2022-08/track_ends_edge_22-07_22-08_cathode_crossers_{cathode_crosser}.npy", ends_at_edges)   
    


[array([[-148.61039105,  108.63300323,  167.60239292],
        [-152.9730072 ,  105.01418404,  173.34479038],
        [-148.53900146,   77.59500122,   27.68812151],
        ...,
        [-152.9730072 ,  130.80299377,   83.15867399],
        [-152.9730072 ,  -59.85900116,   17.89807696],
        [-152.5141066 ,  -42.1230011 ,   43.81342019]])
 array([[147.51137043, -82.02899933,   9.09139467],
        [146.93134273,   6.65100002,  68.85085142],
        [152.9730072 ,  64.02986724,  10.99465888],
        ...,
        [152.4955948 ,   2.33309796, 112.10149908],
        [152.25504944,  11.08500004,  60.33486054],
        [150.46491577, -33.25500107,  34.45154128]])
 array([[ -21.80036563, -152.9730072 ,  143.43779408],
        [ 130.80299377, -152.9730072 ,  142.50431279],
        [-139.67100525, -152.73202785,   12.22214549],
        ...,
        [ 108.63300323, -152.9730072 ,   15.23612697],
        [ 102.48629091, -152.9730072 ,   10.20251935],
        [ 113.49035107, -152.9730072 ,   3

/tmp/ipykernel_38210/3962119430.py:179: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ends_at_edges = np.array(ends_at_edges)


In [44]:
data = np.load(f"data/2022-08/track_ends_edge_22-07_cathode_crossers_{cathode_crosser}.npy",allow_pickle=True)
print(data[5][:,2][data[5][:,2]!=300])

[291.49902098 291.72745301 292.66024354 291.24895448 298.47941895
 297.90786267 317.08874571 292.04975041 291.84230613 294.08213993
 294.46743944 296.73189653 291.87805242 312.2046812  291.72651916
 311.55142537 302.6320907  295.54684671 305.54525361 292.52659339
 292.42923218 291.92108658 293.22135568 295.85792944 298.87119188
 292.85571081 293.1559488  296.26018712 310.61397441 292.91930581
 292.06662308 292.56850348 314.40393206 292.30894908 294.63554453
 294.04155616 291.83811715 291.99987112 293.5249553  294.34954564
 292.95190468 293.18063311 300.39757049 291.63819981 291.35645836
 295.93544189 292.36327739 291.43367104 309.97828215 293.51554411]
